In [1]:
from mqt.yaqs.core.data_structures.networks import MPS, MPO
from mqt.yaqs import simulator
from mqt.yaqs.core.data_structures.simulation_parameters import AnalogSimParams, Observable
from mqt.yaqs.core.libraries.gate_library import X, Z, RuntimeCost
from mqt.yaqs.core.data_structures.noise_model import NoiseModel

import matplotlib.pyplot as plt
import numpy as np
import pickle

In [ ]:
def tdvp_simulator(H_0, noise_model, state=None):
    L = H_0.length

    # Ising
    state = MPS(length=L)

    # Heisenberg
    # elapsed_time = 2
    # state = MPS(length=L, state="Neel")

    measurements = [Observable(Z(), [L//2])] + [Observable("max_bond")] 
    sim_params = AnalogSimParams(observables=measurements,
                                elapsed_time=5,
                                dt=0.1,
                                num_traj=1000,
                                threshold=1e-3,
                                trunc_mode="relative",
                                sample_timesteps=True)

    simulator.run(state, H_0, sim_params, noise_model=noise_model)
    return sim_params.observables

In [ ]:
L_list = [8, 16, 32]
for L in L_list:
    print(L)
    J = 1
    h = 1
    H_0 = MPO()
    H_0.init_ising(L, J, h)

    gamma = 10
    results = []
    # Define the noise model
    noise_model = NoiseModel([
        {"name": name, "sites": [i], "strength": gamma} for i in range(L) for name in ["pauli_z"]
        ])
    # noise_model = NoiseModel([
    #     {"name": name, "sites": [i], "strength": gamma} for i in range(L) for name in ["measure_0", "measure_1"]
    # ])
    cost = tdvp_simulator(H_0, noise_model)
    results.append(cost)

    filename = f"pauli_convergence_{L}.pickle"
    with open(filename, 'wb') as handle:
        pickle.dump(results, handle)

8


Running trajectories:   0%|                           | 0/10000 [00:00<?, ?it/s]

Running trajectories:  43%|██████▊         | 4260/10000 [11:32<24:35,  3.89it/s]